## 60대 데이터 늘리기

In [54]:
import torch
from timm.data.mixup import Mixup
from timm.data.dataset import ImageDataset
from timm.data.loader import create_loader
# import torchvision.transforms
import torchvision
import numpy as np
import os
from enum import Enum
from PIL import Image
import PIL

from torchvision.utils import save_image
import shutil
import random

In [55]:
random.seed(42)     # 일정한 데이터 생성을 확인
data_directory = '/opt/ml/input/data/train/images'

In [56]:
def split_profile_by_gender_60s(profiles):      #성별별로 데이터를 나눕니다
    male,female = [], []        
    for profile in profiles:
        if "Fake" in profile:  # 생성한 데이터를 제외한 데이터들을 반환하기 위해
            continue
        id, gender,species,age = profile.split("_")
        if age == "60":
            if gender == "male":
                male.append(os.path.join(data_directory,profile))
            else:
                female.append(os.path.join(data_directory,profile))
    return [male,female]

In [57]:
profiles = os.listdir(data_directory)
profiles = [profile for profile in profiles if not profile.startswith(".")]
male,female = split_profile_by_gender_60s(profiles)

In [58]:
def make_folder(save_dir):
    if os.path.isdir(save_dir):
        return
    else:
        os.mkdir(save_dir)
        
def not_random_make_fakes_by_gender(gender,profiles,save_dir=data_directory):   # 순서대로 mixup
    length = len(profiles)
    id = 0
    ids = sorted(os.listdir(save_dir))
    if ids:
        id = int(sorted(os.listdir(save_dir))[-1].split("_")[0])
    for i in range(length//2):
        id += 1
        save_profile_dir = f"{id:0>6}_{gender}_Fake_60"
        os.mkdir(os.path.join(save_dir,save_profile_dir))
        j = i+length//2

        make_images(i,j,save_dir,save_profile_dir,profiles)
        
def random_make_fakes_by_gender(gender,profiles,save_dir=data_directory):       #random하게 mixup
    limit = len(profiles)//2
    splited_A = set(random.sample([i for i in range(len(profiles))],limit))
    splited_B = list(set([i for i in range(len(profiles))]) - splited_A)
    splited_A = list(splited_A)
    
    id = 0
    ids = sorted(os.listdir(save_dir))
    if ids:
        id = int(sorted(os.listdir(save_dir))[-1].split("_")[0])
    for i in range(len(splited_A)):
        id += 1
        save_profile_dir = f"{id:0>6}_{gender}_Fake_60"
        os.mkdir(os.path.join(save_dir,save_profile_dir))

        make_images(splited_A[i],splited_B[i],save_dir,save_profile_dir,profiles)

def make_images(profile_i,profile_j,save_dir,save_profile_dir,profiles):    # 만든 이미지 저장
    images = ["incorrect_mask","mask1","mask2","mask3","mask4","mask5","normal"]
    ext = ".jpg"
    # all
    for image in images:
        image_A = np.array(Image.open(os.path.join(profiles[profile_i], image+ext)))//2
        image_B = np.array(Image.open(os.path.join(profiles[profile_j], image+ext)))//2
        new_image = (image_A+image_B)
        img = PIL.Image.fromarray(new_image)
        img.save(os.path.join(save_dir,save_profile_dir,image+ext))    
    

def not_random_make_fake_pics(save_dir=data_directory):     # 랜덤하지 않게 반을 나누어 결합
    make_folder(save_dir)
    not_random_make_fakes_by_gender("male",male,save_dir)
    not_random_make_fakes_by_gender("female",female,save_dir)
    print("Make Done.")

def random_make_fake_pics(save_dir=data_directory):         # 랜덤하게 반을 나누어 결합
    make_folder(save_dir)
    random_make_fakes_by_gender("male",male,save_dir)
    random_make_fakes_by_gender("female",female,save_dir)
    print("Randomly Make Done.")
    
    

In [59]:
def rm_fake_pics(save_dir=data_directory):                  # 만든 fake 디렉토리 전체 제거
    if not os.path.isdir(save_dir):
        print("No folder")
        return
    for fake in [i for i in os.listdir(save_dir) if "Fake" in i]:
        fake_dir = os.path.join(save_dir,fake)
        shutil.rmtree(fake_dir)
    print("Remove Done.")



In [60]:
# # # 비교를 위한 코드입니다. 경로의 Default가 실제 데이터셋 위치입니다.    -> 오류 수정했습니다.
# rm_fake_pics('./not_random')
# rm_fake_pics('./random')
# not_random_make_fake_pics('./random')
# random_make_fake_pics('./random')

In [61]:
rm_fake_pics()
random_make_fake_pics()

Remove Done.
Randomly Make Done.
